In [66]:
import sys
import os
sys.path.append(os.path.abspath('..'))  # Add parent dir to path
from utils.chunker import Chunker
from langchain_community.document_loaders import PyMuPDFLoader

from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document

from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()



True

In [51]:
loader = PyMuPDFLoader("Owners_Manual.pdf")
pages = loader.load()
all_text = " ".join([page.page_content for page in pages])
chunker = Chunker()
chunks = chunker.recursive(all_text)  # Choose one: .fixed_size, .nltk_sentence_chunks, etc.

In [76]:
def embeddings(chunks):
    model = SentenceTransformer("all-MiniLM-L6-v2")
    embed_docs = model.encode([doc.page_content for doc in chunks])
    return embed_docs
print(embeddings(chunks))
embedding_model = HuggingFaceEmbeddings()

docs = [Document(page_content=chunk) for chunk in chunks]
vectorstore = FAISS.from_documents(docs, embedding_model)


/var/folders/kw/n_s0gq250750gnpxmjk5l0qm0000gn/T/ipykernel_37068/3559009047.py:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding_model = HuggingFaceEmbeddings()


In [68]:

from langchain_openai import ChatOpenAI
import os

API_KEY = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    openai_api_key=API_KEY
)



In [78]:
# retriever = vectorstore.as_retriever()
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True  # Optional, helpful for debugging
)




In [80]:
# qa_chain.run("talk about Tesla's starting and powering off?")

response = qa_chain.invoke({"query": "talk about Tesla's starting and powering off?"})
print(response["result"])  # or print(response) to see full output including sources


I don't have specific details on Tesla's starting and powering off procedures from the provided context. However, generally, Tesla vehicles start and power off automatically based on the presence of the key fob or smartphone app. When you enter the vehicle with the key fob or your phone, the car powers on, and when you exit and walk away, it powers off. For precise instructions, it would be best to refer to the vehicle's owner's manual or Tesla's official resources.
